In [ ]:
import sys
sys.path.append("..")

import csaf
from csaf import *
import csaf.utils as csaf_utils
from csaf_f16.systems import F16MultiAgentCentral
from csaf_f16.acas import collision_condition


import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

import random

In [ ]:
from IPython.display import Image

import pathlib

plot_fname = f"pub-sub-plot.png"

# create the target system and check it
my_conf = F16MultiAgentCentral()
my_conf.check()

# plot configuration pub/sub diagram as a file -- proj specicies a dot executbale and -Gdpi is a valid dot
# argument to change the image resolution
my_conf.plot_config(fname=pathlib.Path(plot_fname).resolve(), prog=["dot", "-Gdpi=400"])

# display written file to notebook
Image(plot_fname, height=200)

## System Environments

In [ ]:
def air_collision_condition(ctraces):
        """ground collision premature termination condition
        """
        # get the aircraft states
        sa, sb = ctraces['plant_a']['states'], ctraces['plant_b']['states']
        if sa and sb:
            # look at distance between last state
            return (np.linalg.norm(np.array(sa[-1][9:11]) - np.array(sb[-1][9:11]))) < 100
        

class F16MultiAgentCentralEnv(csaf.SystemEnv):
    """System Environment Definition"""
    system_type = F16MultiAgentCentral
    agents = ["autopilot"]
    

# create an environment and check it
my_env = F16MultiAgentCentralEnv(terminating_conditions_all=air_collision_condition)
my_env.check()

In [ ]:
# set the scenario states
my_env.set_state('plant_a', [500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6000.0, 9.0])
my_env.set_state('plant_b', [500.0, 0.0, 0.0, 0.0, 0.0, np.pi, 0.0, 0.0, 0.0, 6000.0, 0.0, 6000.0, 9.0])

In [ ]:
# collect aircraft states
pstates = []

# send signal of zeros - for this example, send only zeros except for throttle
ctrl_signal = [0., 0.0, 0., 0.7]

# step through simulation and collect f16 states
# StopIteration is thrown when the terminating conditions are achieved
do_sim = True
ctime = 0.0
while do_sim:
    try:
        # send the outputs, can collect the inputs
        ctime, comp_input = my_env.step({"outputs_0": [0., 
                                                       np.cos(10 * ctime) * 2,  
                                                       np.sin(10 * ctime) * 10, 
                                                       0.5], 
                                         "outputs_1": ctrl_signal}) 
        
        # break if outside of time span
        if ctime > 10.0:
            break
        
        # get the states and track them over time
        pstates.append((comp_input))
    
    # stop iteration occurs when the termination conditions are satisfied
    except StopIteration as e:
        do_sim = False
        break
        
    # other errors can occur -- maybe solver error
    except Exception as e:
        print(e)
        raise e
        break
        
pstates = np.array(pstates)

In [ ]:
plt.figure(figsize=(8, 8))
y, x = pstates[:, 9:11].T
plt.plot(x, y, label='Plant A')
y, x = pstates[:, 9+17:11+17].T
plt.plot(x, y, label='Plant B')
plt.axis('equal')
plt.xlabel("East / West Position (ft)")
plt.ylabel("North / South Position (ft)")
plt.title("Plant Positions")
plt.grid()
plt.legend()
plt.show()

## Parallel Simulations

In [ ]:
def task(show_status=False):
    my_sys = F16MultiAgentCentral()
    my_sys.set_state('plant_a', [500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6000.0, 9.0])
    my_sys.set_state('plant_b', [500.0, 0.0, 0.0, 0.0, 0.0, np.pi, 0.0, 0.0, 0.0, 
                                 random.random()*5000, random.random()*400-200, 6000.0, 9.0])
    return my_sys.simulate_tspan((0.0, 20.0), 
                                 terminating_conditions_all=air_collision_condition, 
                                 show_status=show_status)

In [ ]:
task(True)

In [ ]:
from joblib import Parallel, delayed
from time import perf_counter
from contextlib import contextmanager


@contextmanager
def catchtime() -> float:
    """simple way to capture time via a context manager"""
    start = perf_counter()
    yield lambda: perf_counter() - start


# simulate 100 samples using joblib
with catchtime() as t:
    trajsp = Parallel(n_jobs=16)(delayed(task)() for i in range(100))
print(f"[parallel for] Execution Time: {t():.4f} secs")

In [ ]:
for t in trajsp:
    plt.scatter(*t['plant_b'].states[0][9:11], marker='x', c='k')
plt.axis('equal')
plt.xlabel("East / West Position (ft)")
plt.ylabel("North / South Position (ft)")
plt.title("Plant B Initial Position")
plt.grid()
plt.show()